## Credit Card Approval Experiment Tracking

In [1]:
!python -V

Python 3.10.11


In [13]:
import requests
import pickle

import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVR

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from tqdm import tqdm

In [3]:
# Data download.
data = pd.read_csv('data/application_record.csv')
record = pd.read_csv('data/credit_record.csv')

**Data Exploration**

In [4]:
# find all users' account open month.
begin_month=pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min))
begin_month=begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'}) 

new_data=pd.merge(data,begin_month,how="left",on="ID") #merge to record datatrain_raw_data.head()

In [5]:
# Assuming 'record' is your DataFrame containing the 'STATUS' and 'dep_value' columns.
record['dep_value'] = None
record.loc[record['STATUS'].isin(['2', '3', '4', '5']), 'dep_value'] = 'Yes'

cpunt=record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0]='Yes' 
cpunt['dep_value'][cpunt['dep_value'] == 0]='No' 
cpunt = cpunt[['dep_value']]

merge_data=pd.merge(new_data,cpunt,how='inner',on='ID')
merge_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,begin_month,dep_value
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-15.0,No
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-14.0,No
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-29.0,No
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-4.0,No
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-26.0,No


In [47]:
# Assuming 'new_data' is your DataFrame.
# Check for missing values (NaN) in the DataFrame
print(merge_data.isna().sum())

# Drop rows with any NaN values
merge_data.dropna(inplace=True)

# Display the first few rows of the cleaned DataFrame
merge_data.head()

ID                         0
CODE_GENDER                0
FLAG_OWN_CAR               0
FLAG_OWN_REALTY            0
CNT_CHILDREN               0
AMT_INCOME_TOTAL           0
NAME_INCOME_TYPE           0
NAME_EDUCATION_TYPE        0
NAME_FAMILY_STATUS         0
NAME_HOUSING_TYPE          0
DAYS_BIRTH                 0
DAYS_EMPLOYED              0
FLAG_MOBIL                 0
FLAG_WORK_PHONE            0
FLAG_PHONE                 0
FLAG_EMAIL                 0
OCCUPATION_TYPE        11323
CNT_FAM_MEMBERS            0
begin_month                0
dep_value                  0
dtype: int64


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,begin_month,dep_value
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-29.0,No
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-4.0,No
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-26.0,No
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-26.0,No
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-38.0,No


In [6]:
# Now, we remove OCCUPATION_TYPE. As we said, it have too many missed data
# In addition, we will get rid of FLAG_MOBIL, FLAG_WORK_PHONE, FLAG_PHONE and FLAG_EMAIL. 
# These parameters do not affect in any way whether a person is creditworthy or not.

merge_data.drop(['ID','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL'], axis=1, inplace=True) 

In [7]:
# Now, let's convert all non-numeric data to numeric data using LabelEncoder().
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for x in merge_data:
    if merge_data[x].dtypes == 'object':
        merge_data[x] = le.fit_transform(merge_data[x])

In [8]:
data = merge_data.copy()

In [9]:
X = data.iloc[:,1:-1] # X value contains all the variables except labels
y = data.iloc[:,-1] # these are the labels
y

0        0
1        0
2        0
3        0
4        0
        ..
36452    1
36453    1
36454    1
36455    1
36456    1
Name: dep_value, Length: 36457, dtype: int32

In [14]:
# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
oversample = SMOTE()
X_balanced, y_balanced = oversample.fit_resample(X_train, y_train)
X_test_balanced, y_test_balanced = oversample.fit_resample(X_test, y_test)

In [15]:
def process_dataframe(data):
#     data.lpep_dropoff_datetime = pd.to_datetime(data.lpep_dropoff_datetime)
#     data.lpep_pickup_datetime = pd.to_datetime(data.lpep_pickup_datetime)

#     data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
#     data.duration = data.duration.apply(lambda td: td.total_seconds() / 60)
#     data = data[(data.duration >= 1) & (data.duration <= 60)]
    
#     data['PULocationID'].astype(str, copy=False)
#     data['DOLocationID'].astype(str, copy=False)
    
    return data

In [17]:
# X_train = process_dataframe(train_raw_data)[num_features + cat_features]
# X_val = process_dataframe(val_raw_data)[num_features + cat_features] 

# y_train = process_dataframe(train_raw_data)['duration']
# y_val = process_dataframe(val_raw_data)['duration'] 

In [18]:
# X_val.isnull().sum()

trip_distance    0
extra            0
fare_amount      0
PULocationID     0
DOLocationID     0
dtype: int64

## Simple Experimnet 

**Data Preprocessing**

In [16]:
lr = Ridge()
lr.fit(X_balanced, y_balanced)

y_pred = lr.predict(X_test_balanced)

mean_squared_error(y_test_balanced, y_pred, squared=False)

0.4289015372655376

## MLflow tracking

In [17]:
import mlflow

In [18]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("test")

<Experiment: artifact_location='file:///D:/Project Pycharm/creditcard_approval_prediction/mlruns/1', creation_time=1690009378628, experiment_id='1', last_update_time=1690009378628, lifecycle_stage='active', name='test', tags={}>

In [64]:
with mlflow.start_run():
    mlflow.set_tag("workspace", "in_class")
    mlflow.log_param("dataset", "data/")
#     mlflow.log_param("validation_data_name", "green_tripdata_2021-02.parquet")
    
    alpha = 0.99
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_balanced, y_balanced) 
    y_pred = lr.predict(X_test_balanced)

    rmse = mean_squared_error(y_test_balanced, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

## Hyperparameters Optimization

In [56]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [57]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("test")

<Experiment: artifact_location='file:///D:/Project Pycharm/creditcard_approval_prediction/mlruns/1', creation_time=1690009378628, experiment_id='1', last_update_time=1690009378628, lifecycle_stage='active', name='test', tags={}>

In [ ]:
# X_balanced, y_balanced = oversample.fit_resample(X_train, y_train)
# X_test_balanced, y_test_balanced = oversample.fit_resample(X_test, y_test)

In [25]:
train = xgb.DMatrix(X_balanced, label=y_train)
validation = xgb.DMatrix(X_test_balanced, label=y_val)

In [59]:
def objective(params):
    with mlflow.start_run():
        num_boost_round = 500
        early_stopping_rounds = 50
        
        mlflow.log_params(params)
        mlflow.log_param('num_boost_round', num_boost_round)
        mlflow.log_param('early_stopping_rounds', early_stopping_rounds)
        mlflow.log_param('train_data_name', 'green_tripdata_2021-01.parquet')
        mlflow.log_param('validation_data_name', 'green_tripdata_2021-02.parquet')
        mlflow.set_tag('model', 'xgboost')

        booster = xgb.train(
            params = params,
            dtrain = train,
            evals = [(validation, "validation")],
            num_boost_round = num_boost_round,
            early_stopping_rounds = early_stopping_rounds
        )
        
        y_pred = booster.predict(validation)
        rmse = mean_squared_error(y_test_balanced, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
        return {'loss': rmse, 'status': STATUS_OK}

In [60]:
grid_search = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child': hp.loguniform('min_child', -1, 3),
    'seed': 111,
    'objective': 'reg:linear'
}

In [61]:
best_model = fmin(
    fn=objective,
    space=grid_search,
    algo=tpe.suggest,
    max_evals=30,
    trials=Trials()
)

[15:28:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:28:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "min_child" } are not used.

[0]	validation-rmse:0.42838                                                                                            
[1]	validation-rmse:0.40007                                                                                            
[2]	validation-rmse:0.39268                                                                                            
[3]	validation-rmse:0.39489                                                                                            
[4]	validation-rmse:0.39943                                                               

2023/07/22 15:28:48 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.



[15:28:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:28:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "min_child" } are not used.

[0]	validation-rmse:0.43008                                                                                            
[1]	validation-rmse:0.39860                                                                                            
[2]	validation-rmse:0.38280                                                                                            
[3]	validation-rmse:0.37601                                                                                            
[4]	validation-rmse:0.37505                                                               

2023/07/22 15:29:01 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.



[15:29:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:29:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "min_child" } are not used.

[0]	validation-rmse:0.42822                                                                                            
[1]	validation-rmse:0.39970                                                                                            
[2]	validation-rmse:0.39261                                                                                            
[3]	validation-rmse:0.39387                                                                                            
[4]	validation-rmse:0.39921                                                               

...

[472]	validation-rmse:0.24378                                                                                          
[473]	validation-rmse:0.24374                                                                                          
[474]	validation-rmse:0.24371                                                                                          
[475]	validation-rmse:0.24393                                                                                          
[476]	validation-rmse:0.24376                                                                                          
[477]	validation-rmse:0.24374                                                                                          
[478]	validation-rmse:0.24376                                                                                          
[479]	validation-rmse:0.24365                                                                                          
[480]	validation-rmse:0.24356           

2023/07/22 15:35:57 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.



[15:36:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:36:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "min_child" } are not used.

[0]	validation-rmse:0.42855                                                                                            
[1]	validation-rmse:0.40002                                                                                            
[2]	validation-rmse:0.39499                                                                                            
[3]	validation-rmse:0.39632                                                                                            
[4]	validation-rmse:0.40121                                                               

2023/07/22 15:36:11 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.



100%|████████████████████████████████████████████████| 30/30 [07:35<00:00, 15.18s/trial, best loss: 0.2427683651738808]


## Train the Best Model

In [62]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [63]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("test")

<Experiment: artifact_location='file:///D:/Project Pycharm/creditcard_approval_prediction/mlruns/1', creation_time=1690009378628, experiment_id='1', last_update_time=1690009378628, lifecycle_stage='active', name='test', tags={}>

In [ ]:
#We took the best params from the MLflow interface and copien them here

best_params = {
     
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params = best_params,
    dtrain = train,
    evals = [(validation, "validation")],
    num_boost_round = 500,
    early_stopping_rounds = 50,
)

In [ ]:
key="???"
if best_params.get(key):
    print(best_params.get(key))
else:
    print("no value")

In [ ]:
pd.DataFrame.from_dict([best_params])

In [ ]:
y_pred = booster.predict(validation)

rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

## Model Logging 

In [ ]:
with open('models/moodel.bin', 'wb') as f_out:
    pickle.dump(booster, f_out)

In [ ]:
with open('preprocessing/process_dataframe.bin', 'wb') as f_out:
    pickle.dump(process_dataframe, f_out)

In [ ]:
mlflow.set_experiment("test")
with mlflow.start_run():
    best_params = {

    }
    
    mlflow.log_params(best_params)
    mlflow.log_param('train_data_name', 'green_tripdata_2022-01.parquet')
    mlflow.log_param('validation_data_name', 'green_tripdata_2022-02.parquet')
    mlflow.set_tag('model', 'xgboost')
    
    booster = xgb.train(
    params = best_params,
    dtrain = train,
    evals = [(validation, "validation")],
    num_boost_round = 500,
    early_stopping_rounds = 50,
    )
    
    mlflow.xgboost.log_model(booster, artifact_path='mlflow_models')
    mlflow.log_artifact('preprocessing/process_dataframe.bin', artifact_path='preprocessing')
    

## Load Model

In [ ]:
logged_model = 'runs:???'
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
type(loaded_model)

In [ ]:
y_preds = loaded_model.predict(X_val)

In [ ]:
mean_squared_error(y_preds, y_val, squared=False)

In [ ]:
y_preds

In [ ]:
print(loaded_model.metadata.get_model_info())

## Sklearn Models

In [ ]:
mlflow.sklearn.autolog()

for algorithm in (LinearSVR, RandomForestRegressor, GradientBoostingRegressor):
    with mlflow.start_run():
        mlflow.log_param('train_data_name', 'green_tripdata_2022-01.parquet')
        mlflow.log_param('validation_data_name', 'green_tripdata_2022-02.parquet')
        mlflow.log_artifact('preprocessing/process_dataframe.bin', artifact_path='preprocessing')
        model = algorithm()
        model.fit(X_train, y_train)
        
        preds = model.predict(X_val)
        rmse = mean_squared_error(preds, y_val, squared=False)
        mlflow.log_metric("rmse", rmse)
        

## MLflow Client

In [ ]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [ ]:
MLFLOW_URI = "sqlite:///mlflow.db"

In [ ]:
client = MlflowClient(MLFLOW_URI)

In [ ]:
client.list_experiments()

In [ ]:
client.create_experiment(name = 'new-experimet')

In [ ]:
client.list_experiments()

In [ ]:
runs = client.search_runs(
    experiment_ids='1',
    run_view_type=ViewType.ACTIVE_ONLY,
    filter_string='metrics.rmse < 7',
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

In [ ]:
for run in runs:
    print(f"run_id:{run.info.run_id}, metrics:{run.data.metrics['rmse']}")